In [1]:
import numpy as np

class Maze:
    def __init__(self, rows, cols, start, goal, obstacles):
        self.rows = rows
        self.cols = cols
        self.start = start
        self.goal = goal
        self.obstacles = obstacles
        self.state = start

    def reset(self):
        self.state = self.start
        return self.state

    def step(self, action):
        row, col = self.state
        if action == "up" and row > 0 and (row - 1, col) not in self.obstacles:
            self.state = (row - 1, col)
        elif action == "down" and row < self.rows - 1 and (row + 1, col) not in self.obstacles:
            self.state = (row + 1, col)
        elif action == "left" and col > 0 and (row, col - 1) not in self.obstacles:
            self.state = (row, col - 1)
        elif action == "right" and col < self.cols - 1 and (row, col + 1) not in self.obstacles:
            self.state = (row, col + 1)

        done = self.state == self.goal
        reward = 1 if done else 0
        return self.state, reward, done

def value_iteration(maze, gamma, epsilon, max_iterations=1000):
    value_function = np.zeros((maze.rows, maze.cols))

    for _ in range(max_iterations):
        delta = 0
        for row in range(maze.rows):
            for col in range(maze.cols):
                if (row, col) == maze.goal:
                    continue  # Value at the goal state is already 0

                old_value = value_function[row, col]
                new_value = max(
                    [sum(maze.step(action)[1] + gamma * value_function[new_row, new_col]
                         for new_row, new_col in [(row-1, col), (row+1, col), (row, col-1), (row, col+1)]
                         if 0 <= new_row < maze.rows and 0 <= new_col < maze.cols and (new_row, new_col) not in maze.obstacles)
                     for action in ["up", "down", "left", "right"]])

                value_function[row, col] = new_value
                delta = max(delta, abs(new_value - old_value))

        if delta < epsilon:
            break

    # Derive the optimal policy from the computed value function
    policy = np.zeros((maze.rows, maze.cols), dtype=int)
    for row in range(maze.rows):
        for col in range(maze.cols):
            if (row, col) == maze.goal:
                continue
            policy[row, col] = np.argmax(
                [sum(maze.step(action)[1] + gamma * value_function[new_row, new_col]
                     for new_row, new_col in [(row-1, col), (row+1, col), (row, col-1), (row, col+1)]
                     if 0 <= new_row < maze.rows and 0 <= new_col < maze.cols and (new_row, new_col) not in maze.obstacles)
                 for action in ["up", "down", "left", "right"]])

    return value_function, policy

def main():
    maze = Maze(rows=3, cols=3, start=(0, 0), goal=(2, 2), obstacles=[(1, 1)])
    gamma = 0.9
    epsilon = 1e-6

    value_function, policy = value_iteration(maze, gamma, epsilon)

    print("Value Function:")
    print(value_function)
    print("\nOptimal Policy:")
    print(policy)

if __name__ == "__main__":
    main()


Value Function:
[[inf inf inf]
 [inf inf inf]
 [inf inf  0.]]

Optimal Policy:
[[0 0 0]
 [0 0 0]
 [0 0 0]]


<ipython-input-1-a12d3d41e395>:43: RuntimeWarning: overflow encountered in scalar add
  [sum(maze.step(action)[1] + gamma * value_function[new_row, new_col]
<ipython-input-1-a12d3d41e395>:49: RuntimeWarning: invalid value encountered in scalar subtract
  delta = max(delta, abs(new_value - old_value))
